In [692]:
from imp import reload
import prepare_data
reload(prepare_data)
from prepare_data import *

## 1. Get data

In [674]:
combined_df = get_data()

## 2. Data Processing

## 3. Feature engineering

In [675]:
combined_df = parse_dates(combined_df,'datetime')

In [676]:
combined_df = combined_df.drop(['temp','season','minute', 'casual', 'registered','day'], axis = 1) #windspeed

In [677]:
combined_df = target_encoding(combined_df,'weather')

In [678]:
# combined_df = target_encoding(combined_df,'month')
#combined_df = target_encoding(combined_df,'hour')

In [679]:
#combined_df['isSunday'] = (combined_df['weekday'] == 'Sunday') * 1

In [680]:
combined_df = combined_df.drop('weekday', axis = 1)

In [681]:
#combined_df = df_xpercentile(combined_df, 'count', 99) #As per Chebychev's rule, 3std. deviations counts for 99% of the data

In [682]:
combined_df['count'] = np.log(combined_df['count'] + 1)

In [683]:
# cols_to_convert = ['month']
# for col in cols_to_convert:
#     combined_df = df_with_dummies(combined_df,col)

In [684]:
combined_df['year'] = combined_df['year'].map({2011:0, 2012:1})

In [685]:
df, df_test = parse_combined_df(combined_df)

## 4. Build model

In [686]:
X = df.drop('count', axis = 1)
y = df['count']

In [687]:
X_train,X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, test_size = 0.2, random_state = 42)

In [688]:
n_folds = 5
for i in range(0,4):
    model = get_model()[i]
    rmsle = rmsle_cv(model,
                    X,
                    y,
                    n_folds);
    print('The error for model is {}'.format((rmsle)))

The error for model is 0.44605887447003806
[21:24:24] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:24:24] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:24:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:24:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:24:26] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
The error for model is 1.0279010323342577
The error for model is 0.39535431886839184
The error for model is 1.021407688969702


In [ ]:
bestscore, best_search = tuning_model(X,y,2)
#pred_test = best_search.predict(df_test)

In [690]:
finalmodel, error = find_model_error2(get_model()[2], 
                                     X_train,
                                     y_train, 
                                     X_test,
                                     y_test)

In [491]:
pred_test = finalmodel.predict(df_test)


In [635]:
df_test_final = df_test_org[['datetime']].join(pd.DataFrame(np.exp(pred_test)-1, columns=['count']))

In [637]:
df_test_final.to_csv('../data/submission/submission8.csv', index=False)

In [638]:
#df_test_final

In [634]:
pred_test = best_search.predict(df_test)

In [614]:
from sklearn.externals import joblib
joblib.dump(best_search.best_estimator_,'model40percenterror',compress=1)

['model40percenterror']

In [613]:
best_search

GridSearchCV(cv=4, error_score='raise-deprecating',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_sampl...=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'n_estimators': [50, 100, 350, 500], 'alpha': [0.01, 0.1, 0.99], 'learning_rate': [0.01, 0.1, 1]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)